<a href="https://colab.research.google.com/github/jah1994/PyTorchDIA/blob/master/Section_5_1_Real_(EMCCD)_Image_Speed_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook runs the speed tests on real EMCCD images in Section 5.1 of the PyTorchDIA manuscript.

In [ ]:
## CPU specs ##
!cat /proc/cpuinfo

In [ ]:
!cat /proc/meminfo

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#############################################
## required to import custom modules in GC ##
import sys
sys.path.append('/content/drive/My Drive/PyTorchDIA - Speed tests')
##############################################

# grab photutils
#!pip install photutils

# other useful imports
import numpy as np
import os
import glob
from astropy.io.fits import getdata
import time
from scipy.signal import convolve2d
from scipy.ndimage.interpolation import shift
from scipy.stats import norm
import PyTorchDIA_EMCCD
import matplotlib.pyplot as plt
%matplotlib inline

PyTorch version: 1.7.0+cu101


In [ ]:
## load images, reference and master flat ##
## this can take a while in colab

def load_cropped_images(path, crop):

  ## reference
  ref_file = os.path.join(path, 'coll_LOB190560Z_Llr_2019-05-14_00129.fits')
  ref_data = getdata(ref_file, header=True)
  ref, ref_fwhm = ref_data[0], ref_data[1]['FWHM']

  ## master flat
  flat_file = os.path.join(path, 'master_flat.fits')
  master_flat = getdata(flat_file, 0, header=True)[0]

  ## shifts
  shift_info = os.path.join(path, 'LOB190560Z_Shifts.txt')
  shifts = np.genfromtxt(shift_info, delimiter="\t", dtype=str) # filename | xs | ys

  ## crop reference and master flat ##
  ref = ref[crop:ref.shape[0]-crop, crop:ref.shape[1]-crop]
  #master_flat = master_flat[crop:master_flat.shape[0]-crop, crop:master_flat.shape[1]-crop]

  ## ensure dtype=np.float32
  ref = np.array(ref, dtype=np.float32)
  master_flat = np.array(master_flat, dtype=np.float32)

  fnames = []
  images = []
  FWHMs = []
  N_images = []

  for image_file in glob.glob(os.path.join(path, "*coll*")):
    # avoid reference
    if ref_file not in image_file:
      fnames.append(image_file.split('/')[-1])
      print(image_file.split('/')[-1])
      image_data = getdata(image_file, header=True)
      image, header = image_data[0], image_data[1]
      image = image[crop:image.shape[0]-crop, crop:image.shape[1]-crop]
      ## apply any crops to image border ##
      images.append(image)
      FWHMs.append(header['FWHM'])
      N_images.append(header['TOT_IM'])

  # convert to numpy arrays (float32)
  images, FWHMs = np.array(images, dtype=np.float32), np.array(FWHMs)

  print(ref.shape, master_flat.shape, images.shape)
  print('Max x shift:', np.max(shifts[:,2].astype(float)))
  print('Max y shift:', np.max(shifts[:,1].astype(float)))
  print(np.median(FWHMs))

  return images, FWHMs, N_images, fnames, shifts, ref, master_flat

In [ ]:
def extend_image_hw(image, kernel_size):
    image_extended = np.zeros((np.shape(image)[0] + kernel_size - 1,
                             np.shape(image)[1] + kernel_size - 1))
    hwidth = np.int((kernel_size - 1) / 2)
    image_extended[hwidth:image_extended.shape[0]-hwidth,
                   hwidth:image_extended.shape[1]-hwidth] = np.array(image, float)
    return image_extended

def model_image(R, kernel, B0):
    model = convolve2d(R, kernel, mode='same') + B0
    return model

def plot_normalised_residuals(epsilon):
    plt.figure(figsize=(5,5))
    plt.hist(epsilon.flatten(), bins='auto', density=True)
    x = np.linspace(-5, 5, 100)
    plt.plot(x, norm.pdf(x, 0, 1))
    plt.xlim(-5, 5)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.xlabel('Normalised residuals', fontsize=20)
    plt.ylabel('Probability', fontsize=20)
    plt.show();

In [ ]:
## path to images
path = '/content/drive/My Drive/PyTorchDIA - Speed tests/LOB190560Z_CollapsedFrames'

# kernel and image sizes to test performance of optimisation of the robust loss function
#kernel_size = [19, 25]
crops = [150, 100, 50, 15]
kernel_size = [19]

for ks in kernel_size:

  print('\nKernel size:', ks)

  for crop in crops:

    print('Crop:', crop)

    # load cropped images
    images, FWHMs, N_images, fnames, shifts, ref, master_flat = load_cropped_images(path, crop)

    # 'sky' subtract the reference
    ref_sky = np.median(ref)
    print('Sky level [ADU]:', ref_sky)
    ref -= ref_sky

    # information to output to record
    times_to_kernel_solution = []
    total_times = []
    image_FWHMs = []
    image_SNRs = []

    for i, image in enumerate(images):


        out_file_name = 'RealImageSpeedTest_' + str(512 - 2*crop) + '_' + str(ks) + '.txt'
        print('Results recorded in:', out_file_name)

        print('\nImage %d of %d' % (i, len(images)))
        print('FWHM:', FWHMs[i])
        print('N images:', N_images[i])

        # align flat with data image
        # data images were aligned to the reference with an integer pixel shift, avoiding resampling
        # this is fine for the small 45x45 arcseconds^2 FoV for the Danish LI camera
        file_name = fnames[i]
        xs, ys = shifts[:,1][np.where(shifts[:,0] == file_name)][0], shifts[:,2][np.where(shifts[:,0] == file_name)][0]
        print('Aligning flat with (xs, ys) shifts:',xs.astype(int), ys.astype(int))
        flat = shift(master_flat, (ys.astype(int),xs.astype(int)), order=0, cval=0.) # integer shift, order=0
        flat = flat[crop:flat.shape[0]-crop, crop:flat.shape[1]-crop]

        t0 = time.time()
        
        # infer kernel via robust PyTorchDIA code
        print('\n(Robust) PyTorchDIA solution')
        SD_steps = 25000
        kernel, B0 = PyTorchDIA_EMCCD.DIA(ref,
                                          image,
                                          flat,
                                          read_noise = 0.,
                                          ks = ks,
                                          lr_kernel = 1e-3,
                                          lr_B = 1e1,
                                          SD_steps = SD_steps,
                                          Newton_tol = 1e-6,
                                          poly_degree=0,
                                          fast=True,
                                          tol = 1e-9,
                                          max_iterations = SD_steps,
                                          fisher=False,
                                          show_convergence_plots=True)

        total_time = time.time() - t0
        
        # SNR calculation
        # compute model
        ext_ref = extend_image_hw(ref, ks)
        ext_M = model_image(ext_ref, kernel, B0)
        hwidth = np.int((ks - 1) / 2)
        M = ext_M[hwidth:ext_M.shape[0]-hwidth, hwidth:ext_M.shape[1]-hwidth] 

        # compute pixel uncertanties
        gain_CCD = 25.8 # CCD gain
        gain_EM = 300. # EM gain
        G = gain_CCD / gain_EM # Total gain
        excess_noise_factor = 2 # EMCCD fudge factor
        shot_noise = M/(G*flat)
        var_model = excess_noise_factor*shot_noise
        pixel_uncertainties = np.sqrt(var_model) # Noise Model

        # compute normalised residuals
        D = image - M
        epsilon = D / pixel_uncertainties
        #plot_normalised_residuals(epsilon)

        sky = np.median(image)
        SNR = np.sum(image - sky) / np.sqrt(np.sum(pixel_uncertainties**2))
        print('SNR:', SNR)

        # image parameter info
        image_FWHMs.append(FWHMs[i]) 
        image_SNRs.append(SNR)
        
        # time to solution
        total_times.append(total_time)

    # summary stats
    print(len(total_times), len(image_FWHMs), len(image_SNRs))
    print('Median solution time:', np.median(total_times))
    print('Median FWHM:', np.median(image_FWHMs))
    print('Median SNR:', np.median(image_SNRs))

    # write to file
    out = np.vstack((total_times, image_FWHMs, image_SNRs)).T
    print('Output file shape:', out.shape)
    np.savetxt(os.path.join(path, out_file_name), out)    